# Efficiencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import localSettings as ls
import os
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
from scipy.optimize import curve_fit

def gauss(x,mu,sigma,A):
    norm = A/(np.sqrt(2*np.pi)*sigma)
    exp  = np.exp(-((x-mu)**2)/(2*sigma*sigma))
    return norm * exp

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
#if USEBDT:
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward
import pandas as pd

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
import load_data_run123

In [ ]:
def COVARIANCE(n_cv,n_var):
    cov = np.empty([len(n_cv), len(n_cv)])
    cov.fill(0)

    for i in range(len(n_cv)):
        for j in range(len(n_cv)):
            cov[i][j] += (n_var[i] - n_cv[i])*(n_var[j] - n_cv[j])

    frac_cov = np.empty([len(n_cv), len(n_cv)])
    corr = np.empty([len(n_cv), len(n_cv)])

    for i in range(len(n_cv)):
        for j in range(len(n_cv)):
            frac_cov[i][j] =  cov[i][j] / (n_cv[i] * n_cv[j])
            corr[i][j] = cov[i][j] / np.sqrt(cov[i][i] * cov[j][j])
    return cov,frac_cov,corr

In [ ]:
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max",
            "trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

LABELS =  ['pi0','nonpi0']

TRAINVARZP = ['shrmoliereavg','shr_score', "trkfit","subcluster",
              "CosmicIPAll3D","CosmicDirAll3D",
              'secondshower_Y_nhit','secondshower_Y_vtxdist','secondshower_Y_dot','anglediff_Y',
              'secondshower_V_nhit','secondshower_V_vtxdist','secondshower_V_dot','anglediff_V',
              'secondshower_U_nhit','secondshower_U_vtxdist','secondshower_U_dot','anglediff_U',
              "shr_tkfit_2cm_dedx_U", "shr_tkfit_2cm_dedx_V", "shr_tkfit_2cm_dedx_Y",
              "shr_tkfit_gap10_dedx_U", "shr_tkfit_gap10_dedx_V", "shr_tkfit_gap10_dedx_Y",
              "shrMCSMom","DeltaRMS2h","shrPCA1CMed_5cm","CylFrac2h_1cm"]

LABELSZP = ['bkg']

def loadBDT(DF):

    for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_0304_extnumi.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            DF[label+"_score"] = booster.predict(xgb.DMatrix(DF[TRAINVAR]),
                                                 ntree_limit=booster.best_iteration)

    for label, bkg_query in zip(LABELSZP, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_0304_extnumi_vx.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            DF[label+"_score"] = booster.predict(xgb.DMatrix(DF[TRAINVARZP]),
                                                 ntree_limit=booster.best_iteration)
            
    return

In [ ]:
def loadnuevars(df):
    
    INTERCEPT = 0.0
    SLOPE = 0.83

    #if (loadnues):
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])
    df['anglediff_V'] = np.abs(df['secondshower_V_dir']-df['shrclusdir1'])
    df['anglediff_U'] = np.abs(df['secondshower_U_dir']-df['shrclusdir0'])                                                                                                                       
    df["ptOverP"] = df["pt"]/df["p"]
    df["phi1MinusPhi2"] = df["shr_phi"]-df["trk_phi"]
    df["theta1PlusTheta2"] = df["shr_theta"]+df["trk_theta"]
    df['cos_shr_theta'] = np.cos(df['shr_theta'])
    df['cos_trk_theta'] = np.cos(df['trk_theta'])
    df['shr_tkfit_nhits_tot'] = (df['shr_tkfit_nhits_Y']+df['shr_tkfit_nhits_U']+df['shr_tkfit_nhits_V'])                                                                                                                                                                                              
    df['shr_tkfit_2cm_nhits_tot'] = (df['shr_tkfit_2cm_nhits_Y']+df['shr_tkfit_2cm_nhits_U']+df['shr_tkfit_2cm_nhits_V'])                                                                                                                                                              
    df['shr_tkfit_gap10_nhits_tot'] = (df['shr_tkfit_gap10_nhits_Y']+df['shr_tkfit_gap10_nhits_U']+df['shr_tkfit_gap10_nhits_V'])                                                                                                                             
    df.loc[:,'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_Y']
    df.loc[(df['shr_tkfit_nhits_U']>df['shr_tkfit_nhits_Y']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_U']
    df.loc[(df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_Y']) & (df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_U']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_V']
    df.loc[:,'shr_tkfit_gap10_dedx_max'] = df['shr_tkfit_gap10_dedx_Y']
    df.loc[(df['shr_tkfit_gap10_nhits_U']>df['shr_tkfit_gap10_nhits_Y']),'shr_tkfit_gap10_dedx_max'] = df['shr_tkfit_gap10_dedx_U']
    df.loc[(df['shr_tkfit_gap10_nhits_V']>df['shr_tkfit_gap10_nhits_Y']) & (df['shr_tkfit_gap10_nhits_V']>df['shr_tkfit_gap10_nhits_U']),'shr_tkfit_gap10_dedx_max'] = df['shr_tkfit_gap10_dedx_V']
    df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]
    df['electron_e'] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE
    df['protonenergy_corr'] = df['protonenergy']+0.000620/df['protonenergy']-0.001792
    df.loc[(df['protonenergy_corr']>9998.), 'protonenergy_corr'] = 0


    loadBDT(df)

In [ ]:

# which samples are considered for the variations
process = "nue" #"nue" #numu #ccpi0 #ncpi0

# which selection variables to load from the dataframes
selvars = "nue" #numu


In [ ]:
# nue variables

if process == "nue":
    #PATH = '/home/david/data/searchingfornues/v08_00_00_44/'
    PATH = '/Users/cerati/Notebooks/PELEE/root_files/0702/detsyst/'
    mcCV   = 'nue_CV.root'
    mcLYD  = 'nue_LYDown.root'
    mcLYR  = 'nue_LYRayleigh.root'
    mcLYA  = 'nue_LYAttenuation.root'
    mcX    = 'nue_ScaleX.root'
    mcYZ   = 'nue_ScaleYZ.root'
    mcAXZ  = 'nue_AngleXZ.root'
    mcAYZ  = 'nue_AngleYZ.root'
    mcSCE  = 'nue_SCE.root'
    mcdEdX = 'nue_dEdX.root'
    mcR2 = 'nue_R2.root'
    
if process == "numu" or process == "mc":
    PATH = '/Users/cerati/Notebooks/PELEE/root_files/0702/detsyst/'
    mcCV   = 'prodgenie_bnb_nu_overlay_DetVar_CV_reco2_v08_00_00_38_run3b_reco2_reco2.root'
    mcLYD  = 'prodgenie_bnb_nu_overlay_DetVar_LYDown_v08_00_00_37_v2_run3b_reco2_reco2.root'
    mcLYR  = 'prodgenie_bnb_nu_overlay_DetVar_LYRayleigh_v08_00_00_37_run3b_reco2_reco2.root'
    mcLYA  = 'prodgenie_bnb_nu_overlay_DetVar_LYAttenuation_v08_00_00_38_run3b_reco2_reco2.root'
    mcX    = 'prodgenie_bnb_nu_overlay_DetVar_wiremod_ScaleX_v08_00_00_38_run3b_reco2_reco2.root'
    mcYZ   = 'prodgenie_bnb_nu_overlay_DetVar_wiremod_ScaleYZ_v08_00_00_38_run3b_reco2_reco2.root'
    mcAXZ  = 'prodgenie_bnb_nu_overlay_DetVar_WireModAngleXZ_v08_00_00_38_exe_run3b_reco2_reco2.root'
    mcAYZ  = 'prodgenie_bnb_nu_overlay_DetVar_WireModAngleYZ_v08_00_00_38_exe_run3b_reco2_reco2.root'
    mcSCE  = 'prodgenie_bnb_nu_overlay_DetVar_SCE_reco2_v08_00_00_38_run3b_reco2_reco2.root'
    mcdEdX = 'prodgenie_bnb_nu_overlay_DetVar_wiremod_ScaledEdX_v08_00_00_39_run3b_reco2_reco2.root'
    
    
if process == "ccpi0":
    PATH = '/Users/cerati/Notebooks/PELEE/root_files/0702/detsyst/'
    mcCV   = 'prodgenie_cc_pi0_overlay_DetVar_CV_reco2_v08_00_00_38_run3b_reco2_reco2.root'
    mcLYD  = 'prodgenie_cc_pi0_overlay_DetVar_LYDown_v08_00_00_37_run3b_reco2_reco2.root'
    mcLYR  = 'prodgenie_cc_pi0_overlay_DetVar_LYReyleigh_v08_00_00_37_run3b_reco2_reco2.root'
    mcLYA  = ''
    mcX    = 'prodgenie_cc_pi0_overlay_DetVar_wiremod_ScaleX_v08_00_00_38_run3b_reco2_reco2.root'
    mcYZ   = 'prodgenie_cc_pi0_overlay_DetVar_wiremod_ScaleYZ_v08_00_00_38_run3b_reco2_reco2.root'
    mcAXZ  = 'prodgenie_data_cc_pi0_overlay_DetVar_WireModAngleXZ_v08_00_00_38_run3b_reco2_reco2.root'
    mcAYZ  = 'prodgenie_data_cc_pi0_overlay_DetVar_WireModAngleYZ_v08_00_00_38_run3b_reco2_reco2.root'
    mcSCE  = 'prodgenie_cc_pi0_overlay_DetVar_SCE_reco2_v08_00_00_39_run3b_reco2_reco2.root'
    mcdEdX = 'prodgenie_cc_pi0_overlay_DetVar_wiremod_ScaledEdX_v08_00_00_39_run3b_reco2_reco2.root'

if process == "ncpi0":
    PATH = '/Users/cerati/Notebooks/PELEE/root_files/0702/detsyst/'
    mcCV   = 'prodgenie_nc_pi0_overlay_DetVar_CV_reco2_v08_00_00_38_run3b_reco2_reco2.root'
    mcLYD  = 'prodgenie_nc_pi0_overlay_DetVar_LYDown_v08_00_00_37_run3b_reco2_reco2.root'
    mcLYR  = 'prodgenie_nc_pi0_overlay_DetVar_LYReyliegh_v08_00_00_37_run3b_reco2_reco2.root'
    mcLYA  = ''
    mcX    = 'prodgenie_nc_pi0_overlay_DetVar_wiremod_ScaleX_v08_00_00_38_run3b_reco2_reco2.root'
    mcYZ   = 'prodgenie_nc_pi0_overlay_DetVar_wiremod_ScaleYZ_v08_00_00_38_run3b_reco2_reco2.root'
    mcAXZ  = ''
    mcAYZ  = 'prodgenie_nc_pi0_overlay_DetVar_WireModAngleYZ_v08_00_00_38_run3b_reco2_reco2.root'
    mcSCE  = 'prodgenie_nc_pi0_overlay_DetVar_SCE_reco2_v08_00_00_38_run3b_reco2_reco2.root'
    mcdEdX = 'prodgenie_nc_pi0_overlay_DetVar_wiremod_ScaledEdX_v08_00_00_39_run3b_reco2_reco2.root'


In [ ]:
#DETVAR_N_V = ["X", "YZ", 'aYZ', "aXZ"]#,"dEdX","SCE","LYD","LYR","LYA"]
#DETVAR_S_V = [mcX,mcYZ,mcAYZ,mcAXZ]#,nuedEdX,nueSCE,nueLYD,nueLYR,nueLYA]

#DETVAR_N_V = ["dEdX","SCE"]
#DETVAR_S_V = [mcdEdX,mcSCE]

#DETVAR_N_V = ["LYD","LYR","LYA"]
#DETVAR_S_V = [mcLYD,mcLYR,mcLYA]

#DETVAR_N_V = ["X", "YZ", 'aYZ', "aXZ","dEdX","SCE","LYD","LYR","LYA"]
#DETVAR_S_V = [mcX,mcYZ,mcAYZ,mcAXZ,mcdEdX,mcSCE,mcLYD,mcLYR,mcLYA]

#DETVAR_N_V = ["X", "YZ", 'aYZ',"dEdX","SCE","LYD","LYR"] #, "aXZ","LYA"
#DETVAR_S_V = [mcX,mcYZ,mcAYZ,mcdEdX,mcSCE,mcLYD,mcLYR] #,mcAXZ,mcLYA

DETVAR_N_V = ["X", "YZ", 'aYZ',"R2","SCE","LYD","LYR","aXZ","LYA"]
DETVAR_S_V = [mcX,mcYZ,mcAYZ,mcR2,mcSCE,mcLYD,mcLYR,mcAXZ,mcLYA]

In [ ]:
fold = 'nuselection'
tree = 'NeutrinoSelectionFilter'

In [ ]:
VARDICT = load_data_run123.get_variables()

In [ ]:
if (selvars == "nue"):
    variables = VARDICT['VARIABLES'] + VARDICT['WEIGHTS'] + VARDICT['NUEVARS'] #+ load_data_run123.RCVRYVARS
if (selvars == "numu"):
    variables = VARDICT['VARIABLES'] + VARDICT['WEIGHTS'] + VARDICT['CRTVARS']

In [ ]:
# Central Value DF
CV = uproot.open(PATH+mcCV)[fold][tree]
CVDF  = CV.pandas.df(variables, flatten=False)

if (selvars == "nue"):
    load_data_run123.process_uproot(CV,CVDF)
    loadnuevars(CVDF)
if (selvars == "numu"):
    load_data_run123.process_uproot_numu(CV,CVDF)

CVDF['bnbdata']  = np.zeros_like(CVDF['nslice'])
CVDF['extdata']  = np.zeros_like(CVDF['nslice'])


CVDF['identifier'] = CVDF['run']*100000 + CVDF['evt']  + ((100.*CVDF['nu_e']).astype(int))/1000. #to line up events with sample events

NCV = CVDF.shape[0]

print ('there are %i CV events'%(CVDF.shape[0]))

DETSYS_SAMPLE_V = [] #list of merged CV-VAR dfataframes
POT_V = [] #POT of samples? Not really used elsewhere

for i,N in enumerate(DETVAR_N_V):
    
    VAR = uproot.open(PATH+DETVAR_S_V[i])[fold][tree]
    VARDF = VAR.pandas.df(variables, flatten=False)
    
    VARDF['bnbdata']  = np.zeros_like(VARDF['nslice'])
    VARDF['extdata']  = np.zeros_like(VARDF['nslice'])

    
    if (selvars == "nue"):
        load_data_run123.process_uproot(VAR,VARDF)
        loadnuevars(VARDF)
    if (selvars == "numu"):
        load_data_run123.process_uproot_numu(VAR,VARDF)

    VARDF['identifier'] = VARDF['run']*100000 + VARDF['evt']  + ((100.*VARDF['nu_e']).astype(int))/1000.     
    INT = pd.merge(CVDF, VARDF, how='inner', on=['identifier'],suffixes=('_CV', '_VAR'))

    print ('intersection for %15s variation has %i events'%(DETVAR_N_V[i],INT.shape[0]))
    DETSYS_SAMPLE_V.append(INT)
    
    #break

In [ ]:
DETSYS_SAMPLE_V[0]

In [ ]:
def _selection(variable, sample, query="selected==1"):#, extra_cut='',verbose=False):
    """
    variable: string, about which info to be returned
    sample: dataframe, to be queried
    query: string, event-level queries
    extra_cut: string, just another cut in addition to query
    select_longest: bool, will select longest track in each slice after cuts
        should be on when variable is track-level
    fix: string, should be "_CV" or "_VAR"
        on which sample to apply the cuts
    return_fix: string, should be "_CV" or "_VAR"
        which sample to return
    
    Returns an array of track/event variables that pass cuts
    """
        
    df = sample.copy().query(query)
    #start dealing with the track variables
    VARS = df[variable]#VARS is not clean of empty frames
    
    return VARS

In [ ]:
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm

plt.rcParams.update({'font.size': 14})

import unblinding_far_sideband
importlib.reload(unblinding_far_sideband)

import detsysselections
importlib.reload(detsysselections)

In [ ]:
if (selvars == "nue"):

    sdb_key = 'None'
    pre_key = 'NP'
    sel_key = 'NPBDT'
    QUERY_CV  = detsysselections.BDTCQ("CV")
    QUERY_VAR = detsysselections.BDTCQ("VAR")

    #sdb_key = 'None'
    #pre_key = 'ZP'
    #sel_key = 'ZPBDT'
    #QUERY_CV  = detsysselections.ZPBDTCQ("CV")
    #QUERY_VAR = detsysselections.ZPBDTCQ("VAR")

    #sdb_key = 'None'
    #pre_key = 'NP'
    #sel_key = 'NPXSBDT'
    #QUERY_CV  = detsysselections.NPBDTXSQ("CV")
    #QUERY_VAR = detsysselections.NPBDTXSQ("VAR")

    #sdb_key = 'None'
    #pre_key = 'NUE'
    #sel_key = 'XPXSBDT'
    #QUERY_CV  = detsysselections.XPBDTXSQ("CV")
    #QUERY_VAR = detsysselections.XPBDTXSQ("VAR")

    #sdb_key = 'None'
    #pre_key = 'ZP'
    #sel_key = 'ZPXSBDT'
    #QUERY_CV  = detsysselections.ZPBDTXSQ("CV")
    #QUERY_VAR = detsysselections.ZPBDTXSQ("VAR")

    #sdb_key = 'TwoPShr'
    #pre_key = 'NP'
    #sel_key = 'None'
    #QUERY_CV  = detsysselections.NPPRESQ_2pshowers("CV")
    #QUERY_VAR = detsysselections.NPPRESQ_2pshowers("VAR")

    #sdb_key = 'TwoPShr'
    #pre_key = 'ZP'
    #sel_key = 'ZPLOOSETWOSHR'
    #QUERY_CV  = detsysselections.ZPLCUTQ_2pshowers("CV")
    #QUERY_VAR = detsysselections.ZPLCUTQ_2pshowers("VAR")
    
if (selvars == "numu"):
    
    sdb_key = 'None'
    pre_key = 'NUMU'
    sel_key = 'NUMU'

    QUERY_CV  = detsysselections.NUMUPRESEL("CV")
    QUERY_VAR = detsysselections.NUMUPRESEL("VAR")
    
    
sideband = unblinding_far_sideband.sideband_categories[sdb_key]
preselection = unblinding_far_sideband.preselection_categories[pre_key]
sel =  unblinding_far_sideband.selection_categories[sel_key]

SAVEPATH = ls.ntuple_path+'/detsys/'+'{}_{}_{}/'.format(sideband['dir'], preselection['dir'], sel['dir'])

if not os.path.exists(SAVEPATH):
    os.makedirs(SAVEPATH)

In [ ]:
SAVEFIG = True

In [ ]:
if (selvars == "nue"):
    plot_variables  = unblinding_far_sideband.kinematic_variables
    plot_variables += unblinding_far_sideband.energy_variables
    plot_variables += unblinding_far_sideband.bdt_common_variables_1eNp
    plot_variables += unblinding_far_sideband.bdt_1enp_variables
    plot_variables = [['reco_e',7,(0.05, 2.85),r"Reconstructed Energy [GeV]"]]
    plot_variables = [['trkpid',10,(-1,1),"track LLR PID"]]
    plot_variables = [['reco_e',14,(0.15, 1.55),r"reco energy"]]
    plot_variables = [['reco_e',7,(0.1, 1.5),r"reco energy"]]
    plot_variables = [['reco_e',10,(0.15, 1.55),r"reco energy"]]
    #plot_variables = [['cos_trk_theta',6,(-1,1),r"Leading Proton cos($\theta$)"]]
    #plot_variables = [['cos_trk_theta',np.array([-1.,0.,0.4,0.7,1]),None,r"Reconstructed Proton $\cos\left(\theta\right)$"]]
    #plot_variables = [['cos_shr_theta',np.array([-1.,0.2,0.7,0.9,1]),None,r"Reconstructed Electron $\cos\left(\theta\right)$"]]
    #plot_variables = [['protonenergy',np.array([0,0.05,0.1,0.2,0.3,0.8]),None,r"Reconstructed leading proton energy"]]
    #plot_variables = [['protonenergy_corr',np.array([0.,0.05,0.1,0.2,0.3,0.8]),None,r"Reconstructed leading proton energy [GeV]"]]
    #plot_variables = [['electron_e',np.array([0.,0.4,0.9,1.5,4.5]),None,r"Electron Energy [GeV]"]]
    #plot_variables = [['tksh_angle',10,(-1,1),r"cos($\Omega_{e,p}$)"]]
    #plot_variables = [['n_tracks_tot',1,(-0.5,0.5),r"number of track"]]
    #plot_variables = [['tksh_distance',10,(0,10.0),"Conversion distance [cm]"]]
    #plot_variables = [['n_protons_attach',4,(0.5, 4.5),"Number of Protons At Vertex"]]
if (selvars == "numu"):
    #plot_variables  = unblinding_far_sideband.numupresel_variables
    plot_variables = unblinding_far_sideband.numusel_variables

In [ ]:
importlib.reload(ls)

#choose what you want to show
#VARIABLE = 'reco_e'
#TITLE = r'reconstructed neutrino energy [GeV]'
#BINEDGES = np.linspace(0.15,1.55,15)

#print (BINEDGES)
#print(QUERY_CV)

df_Err = []

for VARIABLE,NBINS,RANGE,TITLE in plot_variables:

    print(VARIABLE,NBINS,RANGE,TITLE)
    
    if RANGE is None:
        BINEDGES = NBINS
    else :
        BINEDGES = np.linspace(RANGE[0],RANGE[1],NBINS+1)

    #output errors
    ERROR_OUT_V = []
    QUAD_ERROR_V = np.zeros(len(BINEDGES)-1)

    #fig = plt.figure(figsize=(20,3*len(DETVAR_N_V)))

    index_labels = []
    for b,bin_edge in enumerate(BINEDGES[:-1]):
        index_labels.append("{}-{}".format(round(bin_edge,2),round(BINEDGES[b+1],2)))   
    df_error = pd.DataFrame(index=index_labels) 

    for idx,df_perm in enumerate(DETSYS_SAMPLE_V):
        df = df_perm.copy()
        print("starting {}...".format(DETVAR_N_V[idx]))

        VARS_CV = _selection(VARIABLE+'_CV',df,QUERY_CV)

        VARS_VAR = _selection(VARIABLE+'_VAR',df,QUERY_VAR)

        #######################################
        fig = plt.figure(figsize=(12,5))
        gs = fig.add_gridspec(1, 2)

        axis = fig.add_subplot(gs[0, 0])    
        ################################
        # CV-VAR histogram comparison
        #get queried arrays of the variable
        #get number of entries in each bin for each sample. and plot hists
        n_cv, bins, p = axis.hist(VARS_CV ,bins=BINEDGES,histtype='step',\
                                  lw=2,color='k',label='CV')
        n_var, bins, p = axis.hist(VARS_VAR,bins=BINEDGES,histtype='step',\
                                   lw=2,color='r',label='var : %s'%DETVAR_N_V[idx])

        print ('n CV  : ',n_cv)
        print ('n VAR : ',n_var)

        bc = 0.5*(bins[1:]+bins[:-1]) #bin centers

        cov,frac_cov,corr = COVARIANCE(n_cv,n_var) #calculate various matrices
        error = np.sqrt(np.diag(frac_cov)) #systematic error is this
        
        df_error[DETVAR_N_V[idx]] = error

        ERROR_OUT_V.append(error)
        QUAD_ERROR_V += np.diag(frac_cov)
        ##why Recomb is not added in quad_sum? 
        '''if (DETVAR_N_V[idx] != "Recomb"):
            ERROR_OUT_V.append(error)
            QUAD_ERROR_V += np.diag(frac_cov)'''

        axis.set_xlabel(TITLE)
        axis.set_ylabel('Num. Entries',fontsize=16)
        #plt.ylim(0,plt.gca().get_ylim()[1]*1.5)
        axis.legend(fontsize=15,loc="best")
        #plt.title(SAMPLE)

        ########################################
        axis = fig.add_subplot(gs[0, 1])
        #####################################
        # Fractional Covariance
        #be consistent when comparing multiple plots
        pos = axis.imshow(frac_cov, origin='lower', cmap='viridis',vmin=-0.025,vmax=0.025) 
        #print values onto the plot
        # Limits for the extent
        x_start = 0
        x_end = len(n_cv)#-1
        y_start = 0
        y_end = len(n_cv)#-1
        size = len(n_cv)#-1
        jump_x = (x_end - x_start) / (2.0 * size)
        jump_y = (y_end - y_start) / (2.0 * size)
        x_positions = np.linspace(start=x_start, stop=x_end, num=size, endpoint=False)
        y_positions = np.linspace(start=y_start, stop=y_end, num=size, endpoint=False)
        for x_index, x in enumerate(x_positions):
            #for x_index, x in enumerate(x_positions):
            ERR = frac_cov[x_index, x_index]
            label = "{:.1f}".format(100.*np.sqrt(ERR))
            text_x = x #+ jump_x
            text_y = x #+ jump_y
            if (100.*np.sqrt(ERR) > 8):
                axis.text(text_x, text_y, label, color='black', ha='center', va='center',fontsize=8)
            else:
                axis.text(text_x, text_y, label, color='white', ha='center', va='center',fontsize=8)
        fig.colorbar(pos, ax=axis)
        axis.set_ylabel("Bin number")
        axis.set_xlabel("Bin number")
        axis.set_title(sel_key)


        plt.tight_layout()
        plt.show()

        if (SAVEFIG):
            fig.savefig(SAVEPATH+VARIABLE+"_%s_%s.pdf"%(process,DETVAR_N_V[idx]))

            # diagonal errors
            fout = open(SAVEPATH+VARIABLE+"_%s_%s.txt"%(process,DETVAR_N_V[idx]),'w')
            for i,e in enumerate(error):
                fout.write('%.02f - %.02f, %.03f \n'%(BINEDGES[i],BINEDGES[i+1],e))
            fout.close()
            # covariance matrix
            fout = open(SAVEPATH+VARIABLE+"_%s_%s_cov.txt"%(process,DETVAR_N_V[idx]),'w')
            string = ''
            for n in range(len(error)):
                string += '%.02f-%.02f, '%(BINEDGES[n],BINEDGES[n+1])
            string += '\n'
            fout.write(string)
            for i in range(len(error)):
                string = '%.02f-%.02f, '%(BINEDGES[i],BINEDGES[i+1])
                for j in range(len(error)):
                    string += '%.04f, '%frac_cov[i,j]
                string += '\n'
                fout.write(string)
            fout.close()


    #SAVEFIG = False

    #quadsum
    samples = list(df_error.keys())
    df_error['sum'] = np.sqrt((df_error[samples]**2).sum(axis=1))
    df_Err.append(df_error)
    
    #print ("output Error : ", ERROR_OUT_V)
    #print ("output Error2 : ", np.array(np.sqrt(QUAD_ERROR_V)))


In [ ]:
ax_BINS = plot_variables[0][1]
xtitle = plot_variables[0][3]
labels = ['WireMod X', 'WireMod Y,Z', r'WireMod $\theta_{XZ}$', 'Recomb','SCE', 'LY down', 'LY Rayleigh', r'WireMod $\theta_{YZ}$', 'LYA','total uncertainty' ]

#****add 1st element of the column in the beginning again****
fig, ax1 = plt.subplots(figsize=(10,8))
for i,columns in enumerate(df_error):
    if columns=='sum':
        ax1.plot(ax_BINS, np.append(df_error[columns][0], df_error[columns]), drawstyle='steps-pre', label='total uncertainty',lw=2,color='k',linestyle='--')
    else:
        ax1.plot(ax_BINS, np.append(df_error[columns][0], df_error[columns]), drawstyle='steps-pre', label=labels[i])
ax1.legend(loc=2,ncol=3,frameon=False) 
ax1.set_xlabel(xtitle)
ax1.set_ylabel('Fractional Uncertatinty')
#ax1.set_title(r'$\pi_0$ selection')
ax1.set_ylim(0,0.25)
ax1.set_xlim(ax_BINS[0],ax_BINS[-1])
#fig.savefig("/Users/cerati/Notebooks/PELEE/plots/xsec-nue-bnb/%s_detsyts_summary.pdf"%(plot_variables[0][0]))